# PYTHON ASSIGNMENT CS253
This notebook contains experimentation without 'Total Assets' and 'Liabilities' that were found to have fairly low correlation with 'Education' and had very high range.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing
import sklearn.model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

## IMPORTING LIBRARIES AND DATAFRAME CREATION FRO TRAINING

In [ ]:
df=pd.read_csv('../train.csv')

In [ ]:
le=sklearn.preprocessing.LabelEncoder()
df['state']=le.fit_transform(df['state'])
le_1=sklearn.preprocessing.LabelEncoder()
df['Party']=le_1.fit_transform(df['Party'])
le_2=sklearn.preprocessing.LabelEncoder()
df['Education']=le_2.fit_transform(df['Education'])
df=df.drop(['Candidate','Constituency ∇','Total Assets','Liabilities'],axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   ID             2059 non-null   int64
 1   Party          2059 non-null   int64
 2   Criminal Case  2059 non-null   int64
 3   state          2059 non-null   int64
 4   Education      2059 non-null   int64
dtypes: int64(5)
memory usage: 80.6 KB


In [ ]:
X=df.drop(['ID','Education'],axis=1)
y=df['Education']
X_train,X_test,y_train,y_test=sklearn.model_selection.train_test_split(X,y,test_size=0.1,random_state=42)

## DECISION TREE CLASSIFIER

In [ ]:
param_grid = {'max_depth':range(1,20),'max_leaf_nodes':[None,1,2,3,4,5,6,7,8,9,10],'min_samples_split':[2,3,4,5,6,7,8,9,10],'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10]}
dt_model = DecisionTreeClassifier()
dt_model_cv = GridSearchCV(dt_model,param_grid=param_grid,cv=10,scoring='f1_weighted')
dt_model_cv.fit(X_train,y_train)

/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(
/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
17100 fits failed out of a total of 188100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
17100 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/aditikhandelia/anaconda3/lib/python3

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 20),
                         'max_leaf_nodes': [None, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                            10],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='f1_weighted')

In [ ]:
print('Best Parameters:',dt_model_cv.best_params_)
y_pred=dt_model_cv.predict(X_test)
print('Accuracy:',accuracy_score(y_test,y_pred))
print('Precision:',precision_score(y_test,y_pred,average='weighted'))
print('Recall:',recall_score(y_test,y_pred,average='weighted'))
print('F1:',f1_score(y_test,y_pred,average='weighted'))

Best Parameters: {'max_depth': 14, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Accuracy: 0.2524271844660194
Precision: 0.22970790300788924
Recall: 0.2524271844660194
F1: 0.2401183976703066


/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


The F1 score on test split is quite high. Thus, the model trained is used to create a submission.csv on test dataset.

In [ ]:
testing_df_final=pd.read_csv('../test.csv')
testing_df_final['state']=le.transform(testing_df_final['state'])
testing_df_final['Party']=le_1.transform(testing_df_final['Party'])
testing_df_final=testing_df_final.drop(['Candidate','Constituency ∇','Total Assets','Liabilities'],axis=1)
testing_df_final_x=testing_df_final.drop(['ID'],axis=1)
testing_df_final_y_pred=dt_model_cv.predict(testing_df_final_x)
testing_df_final_y_pred=le_2.inverse_transform(testing_df_final_y_pred)
testing_df_final['Education']=testing_df_final_y_pred
testing_df_final=testing_df_final[['ID','Education']]
testing_df_final.to_csv('submission_decision_tree.csv',index=False)

## KNN CLASSIFIER

In [ ]:
param_grid={'n_neighbors':range(1,30)}
knn_model=sklearn.neighbors.KNeighborsClassifier()
knn_model_cv=GridSearchCV(knn_model,param_grid=param_grid,cv=10,scoring='f1_weighted')
knn_model_cv.fit(X_train,y_train)

/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(


GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 30)}, scoring='f1_weighted')

In [ ]:
print('Best Parameters:',knn_model_cv.best_params_)
y_pred=knn_model_cv.predict(X_test)
print('Accuracy:',accuracy_score(y_test,y_pred))
print('Precision:',precision_score(y_test,y_pred,average='weighted'))
print('Recall:',recall_score(y_test,y_pred,average='weighted'))
print('F1:',f1_score(y_test,y_pred,average='weighted'))

Best Parameters: {'n_neighbors': 25}
Accuracy: 0.22815533980582525
Precision: 0.16736907740212842
Recall: 0.22815533980582525
F1: 0.19017829321858612


/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Since the result on the test split is quite low, the model needn't be tried on the test dataset

## RANDOM FOREST CLASSIFIER

In [ ]:
param_grid={'max_depth':[3,5,10,None],
              'n_estimators':[10,100,200],
              'max_features':[1,3,5,7],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[1,2,3]
           }
rf_model=RandomForestClassifier()
rf_model_cv=GridSearchCV(rf_model,param_grid=param_grid,cv=10,scoring='f1_weighted')
rf_model_cv.fit(X_train,y_train)

/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(
/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
1440 fits failed out of a total of 4320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1440 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/aditikhandelia/anaconda3/lib/python3.11/

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 5, 10, None],
                         'max_features': [1, 3, 5, 7],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [1, 2, 3],
                         'n_estimators': [10, 100, 200]},
             scoring='f1_weighted')

In [ ]:
print('Best Parameters:',rf_model_cv.best_params_)
y_pred=rf_model_cv.predict(X_test)
print('Accuracy:',accuracy_score(y_test,y_pred))
print('Precision:',precision_score(y_test,y_pred,average='weighted'))
print('Recall:',recall_score(y_test,y_pred,average='weighted'))
print('F1:',f1_score(y_test,y_pred,average='weighted'))

Best Parameters: {'max_depth': 10, 'max_features': 1, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 10}
Accuracy: 0.24757281553398058
Precision: 0.19685172966349326
Recall: 0.24757281553398058
F1: 0.20837024254571343


/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


The F1 score is higher than the KNN model before.

In [ ]:
testing_df_final=pd.read_csv('./test.csv')
testing_df_final['state']=le.transform(testing_df_final['state'])
testing_df_final['Party']=le_1.transform(testing_df_final['Party'])
testing_df_final=testing_df_final.drop(['Candidate','Constituency ∇','Total Assets','Liabilities'],axis=1)
testing_df_final_x=testing_df_final.drop(['ID'],axis=1)
testing_df_final_y_pred=rf_model_cv.predict(testing_df_final_x)
testing_df_final_y_pred=le_2.inverse_transform(testing_df_final_y_pred)
testing_df_final['Education']=testing_df_final_y_pred
testing_df_final=testing_df_final[['ID','Education']]
testing_df_final.to_csv('submission_random_forest.csv',index=False)

## LINEAR SVC

In [ ]:
param_grid={'C':[0.1,1,10,100,1000],'gamma':[1,0.1,0.01,0.001,0.0001],'kernel':['rbf']}
svc_model=SVC()
svc_model_cv=GridSearchCV(svc_model,param_grid=param_grid,cv=10,scoring='f1_weighted')
svc_model_cv.fit(X_train,y_train)

/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(


GridSearchCV(cv=10, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             scoring='f1_weighted')

In [ ]:
print('Best Parameters:',svc_model_cv.best_params_)
y_pred=svc_model_cv.predict(X_test)
print('Accuracy:',accuracy_score(y_test,y_pred))
print('Precision:',precision_score(y_test,y_pred,average='weighted'))
print('Recall:',recall_score(y_test,y_pred,average='weighted'))
print('F1:',f1_score(y_test,y_pred,average='weighted'))

Best Parameters: {'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy: 0.23300970873786409
Precision: 0.20454968557543482
Recall: 0.23300970873786409
F1: 0.21357953863401283


/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
